# **Lenguajes de Programación - Tutorial Erlang**

# Ejemplos

Autores:
*   Julián Alexander Manosalva Manrique
*   Juan Sebastián Pachón Carvajal
*   David Alexander Zambrano Bohórquez





In [ ]:
!!apt install erlang

**Ejemplo 0**

Escribir un programa en Erlang que genere dos procesos concurrentes uno que sume y otro que reste dos números.

In [ ]:
%%writefile arithmetic.erl

-module(arithmetic).
-export([start/0, sum/2, subtract/2]).

start() ->
    spawn(arithmetic, sum, [8, 2]),
    spawn(arithmetic, subtract, [8, 2]).

sum(A, B) ->
    Result = A + B,
    io:format("Sum result: ~w~n", [Result]).

subtract(A, B) ->
    Result = A - B,
    io:format("Subtraction result: ~w~n", [Result]).

Writing arithmetic.erl


In [ ]:
!erlc arithmetic.erl
!erl -noshell -s arithmetic start -s init stop

Sum result: 10
Subtraction result: 6


**Ejemplo 1**




Implementar un programa en Erlang, que implemente un contador ascendente y otro descendente. Luego editar este módulo para que los contadores se ejecuten de manera concurrente,

In [ ]:
%%writefile counter_lineal.erl

-module(counter_lineal).
-export([start/0]).


start_counting(N) ->
  count_up(N, 1),
  count_down(N, N).

count_up(N, Current) ->
  io:format("Contador 1: ~w~n", [Current]),
  if
    Current < N ->
      count_up(N, Current + 1);
    true ->
      io:format("Ha finalizado el Contador ascendente~n")
  end.

count_down(N, Current) ->
  io:format("Contador 2: ~w~n", [Current]),
  if
    Current > 1 ->
      count_down(N, Current - 1);
    true ->
      io:format("Ha finalizado el Contador descendiente~n")
  end.

start() ->
  start_counting(10).

Overwriting counter_lineal.erl


In [ ]:
!erlc counter_lineal.erl
!erl -noshell -s counter_lineal start -s init stop

Contador 1: 1
Contador 1: 2
Contador 1: 3
Contador 1: 4
Contador 1: 5
Contador 1: 6
Contador 1: 7
Contador 1: 8
Contador 1: 9
Contador 1: 10
Ha finalizado el Contador ascendente
Contador 2: 10
Contador 2: 9
Contador 2: 8
Contador 2: 7
Contador 2: 6
Contador 2: 5
Contador 2: 4
Contador 2: 3
Contador 2: 2
Contador 2: 1
Ha finalizado el Contador descendiente


In [ ]:
%%writefile counter.erl

-module(counter).
-export([start/0]).

start_counting(N) ->
  spawn(fun() -> count_up(N, 1) end),
  spawn(fun() -> count_down(N, N) end).

count_up(N, Current) ->
  io:format("Contador 1: ~w~n", [Current]),
  if
    Current < N ->
      count_up(N, Current + 1);
    true ->
      io:format("Ha finalizado el Contador ascendente~n")
  end.

count_down(N, Current) ->
  io:format("Contador 2: ~w~n", [Current]),
  if
    Current > 1 ->
      count_down(N, Current - 1);
    true ->
      io:format("Ha finalizado el Contador descendiente~n")
  end.

start() ->
  start_counting(10).

Overwriting counter.erl


In [ ]:
!erlc counter.erl
!erl -noshell -s counter start -s init stop

Contador 1: 1
Contador 2: 10
Contador 1: 2
Contador 2: 9
Contador 1: 3
Contador 2: 8
Contador 1: 4
Contador 2: 7
Contador 1: 5
Contador 2: 6
Contador 1: 6
Contador 2: 5
Contador 1: 7
Contador 2: 4
Contador 1: 8
Contador 2: 3
Contador 1: 9
Contador 2: 2
Contador 1: 10
Contador 2: 1
Ha finalizado el Contador ascendente
Ha finalizado el Contador descendiente


**Ejemplo 2**

 Implementar un programa en Erlang que permita calcular el mínimo y el máximo de una lista de números de forma concurrente. El programa debe crear dos procesos: uno para calcular el mínimo y otro para calcular el máximo. Cada proceso debe realizar sus cálculos de manera independiente y enviar el resultado al proceso principal. El proceso principal debe imprimir los valores mínimo y máximo una vez que los haya recibido de los procesos secundarios.

In [ ]:
%%writefile min_max_concurrent.erl
-module(min_max_concurrent).

% Exportamos la función calculate_min_max/1 desde el módulo min_max_concurrent. 
% El número 1 entre paréntesis indica que la función tiene un aridad de 1, lo que significa que toma un argumento.
-export([calculate_min_max/1, start/0]).

% Función principal para calcular el mínimo y el máximo de una lista de forma concurrente (uso de cláusulas).
calculate_min_max(List) ->
    % Obtenemos el identificador del proceso principal.
    ParentPid = self(),
    % Creamos dos procesos secundarios para calcular el mínimo y el máximo.
    % Utilizamos spawn para crear procesos y fun para definir funciones anónimas
    spawn(fun() -> calculate_min(ParentPid, List) end),
    spawn(fun() -> calculate_max(ParentPid, List) end),
    % Esperamos a recibir los resultados de los procesos secundarios.
    receive
        % Recibimos el mensaje con el mínimo.
        {min, Min} ->
            receive
                % Recibimos el mensaje con el máximo.
                {max, Max} ->
                    % Imprimimos el valor mínimo.
                    io:format("Minimum: ~w~n", [Min]),
                    % Imprimimos el valor máximo.
                    io:format("Maximum: ~w~n", [Max])
            end
    end.

% Función para calcular el mínimo de una lista (uso de cláusulas).
% Esta función se encarga de recorrer la lista de números y actualizar el mínimo en cada iteración, manteniendo el valor mínimo actualizado hasta ese punto.
calculate_min(ParentPid, [Head|Tail]) ->
    % Llamamos a la función auxiliar con el valor mínimo actualizado.
    calculate_min(ParentPid, Tail, Head).

% Esta función itera sobre la lista de números y actualiza el valor mínimo (MinSoFar) cada vez que encuentra un número menor. 
% Utiliza la recursión para seguir avanzando en la lista hasta llegar al final y enviar el resultado al proceso principal.
calculate_min(ParentPid, [Head|Tail], MinSoFar) ->
    % Comparamos el número actual con el mínimo actual.
    if
        % Si encontramos un número menor, actualizamos el mínimo.
        Head < MinSoFar ->
            calculate_min(ParentPid, Tail, Head);
        % Si no, continuamos con el mínimo actual.
        true ->
            calculate_min(ParentPid, Tail, MinSoFar)
    end;

% Esta función se ejecuta cuando hemos recorrido toda la lista de números y ya no quedan más elementos para comparar. 
% En este punto, tenemos el valor mínimo final y estamos listos para enviarlo al proceso principal.
calculate_min(ParentPid, [], Min) ->
    % Enviamos el resultado al proceso principal.
    ParentPid ! {min, Min}.

% Función para calcular el máximo de una lista.
calculate_max(ParentPid, [Head|Tail]) ->
    % Llamamos a la función auxiliar con el valor máximo actualizado.
    calculate_max(ParentPid, Tail, Head).

calculate_max(ParentPid, [Head|Tail], MaxSoFar) ->
    % Comparamos el número actual con el máximo actual.
    if
        % Si encontramos un número mayor, actualizamos el máximo.
        Head > MaxSoFar ->
            calculate_max(ParentPid, Tail, Head);
        % Si no, continuamos con el máximo actual.
        true ->
            calculate_max(ParentPid, Tail, MaxSoFar)
    end;
calculate_max(ParentPid, [], Max) ->
    % Enviamos el resultado al proceso principal.
    ParentPid ! {max, Max}.

start() ->
  min_max_concurrent:calculate_min_max([5, 8, 2, 1, 9, 4, 12, 1]).



Writing min_max_concurrent.erl


Para efectos del tutorial, este ejemplo se realizó en Repl.it, una plataforma en línea que proporciona un entorno de desarrollo integrado (IDE) basado en la nube para ejecutar y programar en varios lenguajes de programación, incluido Erlang. Aquí se tiene cómo ejecutar el código anterior en el shell de Repl.it para Erlang:

1.   Abrir un terminal o una línea de comandos en Repl.it.
2.   Iniciar el shell de Erlang escribiendo '*erl*' y presionando Enter.
3.   En el shell de Erlang, compila el código escribiendo '*c(min_max_concurrent).*' y presionando Enter.
4.   Ejecutar el cálculo del mínimo y máximo de una lista llamando a la función *min_max_concurrent:calculate_min_max/1* con una lista de números como argumento. Por ejemplo, escribir '*min_max_concurrent:calculate_min_max([5, 8, 2, 1, 9, 4, 12, 1]).*' y posteriormente presionar Enter.

El programa hallará el valor mínimo y el valor máximo de la lista de forma concurrente y los mostrará en consola, de la siguiente manera:

In [ ]:
!erlc min_max_concurrent.erl
!erl -noshell -s min_max_concurrent start -s init stop

Minimum: 1
Maximum: 12


**Ejemplo 3**

Implementar un programa en erlang que permita organizar una cierta lista dada utilizando el algoritmo merge sort usando concurrencia, utilizando recursión. En este caso cada vez que la lista se divida en dos se pueden crear nuevos procesos. Luego el primero proceso es el que debe imprimir la lista.

Plus: Implementar un programa que verifique el tiempo que toma en realizar el merge sort con y sin concurrencia

In [ ]:
%%writefile parallelMergeSort.erl
-module(parallelMergeSort).
-import(io,[fwrite/1]).
-export([ms/1, pms/1, p_ms/2, start/0]).

start() ->
    seed(),
    List = create_list( 50000, 100000 ),
    { Time1, _ } = timer:tc( parallelMergeSort, ms, [List] ),
    io:format( "Merge sort took ~p seconds.~n", [Time1 / 1000000] ),
    { Time2, _ } = timer:tc( parallelMergeSort, pms, [List] ),
    io:format( "Parallel merge sort took ~p seconds.~n", [Time2 / 1000000] ).

%% seed( ) will seed the random number generator.
seed( ) ->
    rand:seed( exsss, { erlang:phash2( node( ) ), erlang:monotonic_time( ), erlang:unique_integer( ) } ).

%% create_list( Length, Max ) returns a list with Length items in it, all of which are random numbers between 1 and Max.
create_list( 0, _ ) -> [];
create_list( Length, Max ) ->
    [ rand:uniform( Max ) | create_list( Length - 1, Max ) ].

% SEPARATE INTO TWO LISTS USING N AS THE LENGTH OF THE FIRST LIST,
% AND THUS, THE SECOND LIST WILL HAVE length(L)-N elements

% case we say the length of the list1 is 0
sep(L, 0) -> 
	{[], L};

sep([H|T],N) ->
   {Lleft, Lright} = sep(T,N-1),
   	{append([H],Lleft), Lright}.   

append(L1,L2) ->
	L1++L2.


% MERGE RETURNING SORTED LIST OF TWO LISTS
merge(L1,L2) ->
	merge(L1,L2,[]).

merge([], [], RESULT_LIST) -> RESULT_LIST;
merge([], SECOND_LIST, RESULT_LIST) -> append(RESULT_LIST, SECOND_LIST);
merge(FIRST_LIST, [], RESULT_LIST) -> append(RESULT_LIST, FIRST_LIST);
merge([H1|FIRST_LIST], [H2|SECOND_LIST], RESULT_LIST) ->
	if 	H1 < H2 ->
			merge(FIRST_LIST, [H2|SECOND_LIST], append(RESULT_LIST, [H1]));
		true ->	
			merge([H1|FIRST_LIST], SECOND_LIST, append(RESULT_LIST, [H2]))
	end.	

% MERGE SORT IMPLEMENTATION
ms([]) -> [];
ms([X]) -> [X];
ms(L) ->
   {L1, L2} = sep(L, length(L) div 2),
   merge(ms(L1), ms(L2)).


% PARALLEL VERSION OF MERGE SORT
rcvp(Pid) -> 
	receive
		{Pid, L} -> L
	end.

pms(L) ->
	Pid = spawn(parallelMergeSort, p_ms, [self(), L]),
  rcvp(Pid).

p_ms(Pid, L) when length(L) < 100 -> 
  Pid ! {self(), ms(L)}; %Do mergesort
p_ms(Pid, L) -> %Else, if the length is "long", spawn processes
	{Lleft, Lright} = sep(L, length(L) div 2),
    Pid1 = spawn(parallelMergeSort, p_ms, [self(), Lleft]),
    Pid2 = spawn(parallelMergeSort, p_ms, [self(), Lright]),
    L1 = rcvp(Pid1),
    L2 = rcvp(Pid2),
    Pid ! {self(), merge(L1,L2)}. %Send to father the merged lists

Writing parallelMergeSort.erl


In [ ]:
!erlc parallelMergeSort.erl
!erl -noshell -s parallelMergeSort start -s init stop

Merge sort took 7.325266 seconds.
Parallel merge sort took 8.730769 seconds.
